#**POST-DISRUPTION MODELING FOR RESILIENCE IMPROVEMENT** 

Continuous-Time Markov Decision Process (CTMDP) model for improving post-disruption resilience (system's recovery). 

0. Libraries
1. Input files (generator matrices)
2. Model functions
3. Value iteration
4. Policy simulation

# (0) Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import numpy as np
import statsmodels.api as sm
import scipy.stats as sps
import pandas as pd

# (1) Importing input files

In [ ]:
file = files.upload()

# (2) Model functions

## Uniformization model function

Description: The uniformization algorithm is a technique to retrieve the probability transition matrices from the generator (rates) matrices.

Inputs: Generator matrix (Q)

$P = I + (1/\gamma)*Q$

Output: Transition/Stochastic matrix (P)


In [ ]:
#Uniformization function
def prob (rate_matrix):
  Q = rate_matrix
  I = np.identity(len(Q))
  gamma = max(abs(np.diagonal(Q)))
  P = I + (1/gamma)*Q
  return P

In [ ]:
#gamma function
def gamma (rate_matrix):
  gamma = max(abs(np.diagonal(rate_matrix)))
  return gamma

In [ ]:
# Reading all transition matrices
m = 5
Probabilities_matrix = []
Rates_matrix = []
gamma_array = []
for i in range(m):
  name = 'rates_matrix_action_{}.csv'
  instance = pd.read_csv(name.format(i), header=None)
  Probabilities_matrix.append(prob(instance))
  Rates_matrix.append(instance)
  gamma_array.append(gamma(instance)) # Create gammas array for discounted model

## Actions set

In [ ]:
#Actions dictionary
action_set =('ss', 'sm', 'mm', 'ml', 'll') # actions names
a_n = (0,1,2,3,4) # actions numbers a= {0,1,2,3,4}
action_dict ={}

#Populating atcion dictionary
for i in a_n:
  action_dict[a_n[i]] = action_set[i]

## Costs function, $r(s,a)$
$r(s,a) = k(s,a) + c(s,a)*E[\Delta t]$

$E[∆t] = \frac{|S|}{\sum(mu_{ij})}$ for each $i \in S $

In [ ]:
#k(s,a) |S| x |a|
fixed_cost_matrix = pd.read_csv('costs.csv', header=None)

In [ ]:
#Expected transition sojurn time
expected_times_matrix = pd.read_csv('expected_times.csv', header=None)


In [ ]:
#Recovery cost rates
cost_rate = pd.read_csv('costs_rates.csv', header = None)

In [ ]:
c_matrix = [np.array(fixed_cost_matrix[0]+cost_rate[0]*expected_times_matrix[0]), np.array(fixed_cost_matrix[1]+cost_rate[0]*expected_times_matrix[1]), np.array(fixed_cost_matrix[2]+cost_rate[0]*expected_times_matrix[2]), np.array(fixed_cost_matrix[3]+cost_rate[0]*expected_times_matrix[3]), np.array(fixed_cost_matrix[4]+cost_rate[0]*expected_times_matrix[4])]

#Expected recovery costs, r(s,a)
cost_matrix = pd.DataFrame(np.transpose(c_matrix))
cost_matrix.replace([np.inf, -np.inf], 0, inplace=True)

## Data strcuture and preparation
### Tupple -> (trans probs, next state, costs)
trans prob -> (action, next state, current state)

next state -> (i->j)

costs/rewards -> (action, state)


In [ ]:
p = Probabilities_matrix # p -> list of transition probabilities matrices
c = cost_matrix # c -> cost matrix

# Creating lists
list1= []
list2= []
for s in range(len(p[0])): 
  array_2 = []
  for a in range(len(a_n)):
    array_1 = []
    for j in range(len(p[0])):    
      array_1.append((p[a][j][s],j,c[a][s]))
    array_2.append(array_1)
    list1.append(array_1)
  list2.append(array_2)

In [ ]:
#Population directory
cs =[]
for i in range(60):
  cs.append(i)
  
states_dir = {}
for i in range(len(p[0])):
  states_dir[cs[i]] = list2[i]    

# (3) Solution method: Value iteration algorithm

Optimization objective: Costs minimization

### Value iteration function

In [ ]:
# value iteration function
# inputs for value function -> number of states, number of actions, tupple
def value_iteration (n_states, n_actions, P, alpha =.00005):
  value_table = np.zeros(n_states)
  no_of_iterations = 100000
  epsilon = 1e-50
  for i in range(no_of_iterations):
    updated_value_table = np.copy(value_table)
    for state in range(n_states):
      Q_value = []
      for action in range(0, n_actions):
        next_states_rewards = []
        for next_sr in P[state][action]:
          trans_prob, next_state, reward_prob = next_sr
          next_states_rewards.append((trans_prob*(reward_prob + (gamma_array[action]/(gamma_array[action]+alpha)) * updated_value_table[next_state])))
        Q_value.append(np.sum(next_states_rewards))
      value_table[state] = min(Q_value)
    if (np.sum(np.fabs(updated_value_table-value_table)) <= epsilon) :
      print ('Value-iteration converged at iteration # %d.' %(i+1))
      break
    #print ('iteration # %d.' %(i+1))
  return value_table


Now, we define a function called extract policy for extracting optimal policy from the optimal value function. i.e We calculate Q value using our optimal value function and pick up the actions which has the highest Q value for each state as the optimal policy.

In [ ]:
def extract_policy(value_table, n_action, P, alpha =.00005):
  # initialize the policy with zeros
  policy = np.zeros(len(value_table))   
  for state in range(len(value_table)):
  # initialize the Q table for a state
    Q_table = np.zeros(n_action) 
    # compute Q value for all ations in the state
    for action in range(n_action):
      for next_sr in P[state][action]: 
        trans_prob, next_state, reward_prob = next_sr 
        Q_table[action] += (trans_prob * (reward_prob + (gamma_array[action]/(gamma_array[action]+alpha)) * value_table[next_state]))
    # select the action which has maximum Q value as an optimal action of the state
    policy[state] = np.argmin(Q_table)  
  return policy

## Step 1: Compute optimal value function

In [ ]:
number_actions = len(a_n) # number of actions
number_states = len(p[0]) # number of states
optimal_value_function = value_iteration(number_states, number_actions, states_dir)

Value-iteration converged at iteration # 2078.


## Step 2: Derive optimal policy from optimal value function

In [ ]:
optimal_policy = extract_policy(optimal_value_function, number_actions, states_dir)

In [ ]:
#print( "Optimal value function ->", optimal_value_function)
print( "Optimal policy ->", optimal_policy)

Optimal policy -> [1. 1. 1. 1. 1. 1. 2. 2. 2. 3. 0. 1. 1. 1. 1. 1. 3. 3. 3. 3. 1. 1. 1. 1.
 2. 2. 3. 3. 3. 3. 1. 1. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Optimal policy simulation

In [ ]:
#Uniform transition rates
m = 5
exp_rates = [] #exponential rates after removing \inf

for i in range(m):
  name = 'time_rates_matrix_{}.csv'
  r = pd.read_csv(name.format(i), header=None)
  exp_rates.append(r)

In [ ]:
def state_generator(rate, i):
  rv_vector = []
  for j in range(len(rate)):
    rv_vector.append(np.random.exponential(rate[j].loc[i]))
  time = min(i for i in rv_vector if i > 0)
  return rv_vector.index(time)

# (4) Policy comparison simulation

In [ ]:
total_budget = 225.565034 # 225.56 M USD -> 576,000 M COP

# Number of policies to compare
policies_comparison = 0

# Mean recovery time vectors
fst_policy_time = []
opt_policy_time = []
chp_policy_time = []

# Mean recovery cost vectors
fst_policy_cost = []
opt_policy_cost = []
chp_policy_cost = []

# Mean budget utilization vectors
fst_policy_bud = []
opt_policy_bud = []
chp_policy_bud = []

while policies_comparison <= 2:
  # Number of runnings 
  n_of_interations = 0
  while n_of_interations <= 100000 :
  
    recovery_cost = 0
    budget_ut = 0
    time = 0
    state = 0
    current_state_vector = [0]
    next_state_vector = [0]

    # Time vectors per policy
    time_vector_fst = [0]
    time_vector_opt = [0]
    time_vector_chp = [0]

    # Costs vectors per policy
    cost_vector_fst = [0]
    cost_vector_opt = [0]
    cost_vector_chp = [0]

    # Budget utilization vectors per policy
    budget_fst = [0]
    budget_opt = [0]
    budget_chp = [0]
    
    if policies_comparison == 0:
      while  state < (len(optimal_policy)-10):
        # state transition
        decision = 0 # fastest policy   
        next_state = state_generator(exp_rates[decision],state)
        
        current_state_vector.append(state)
        next_state_vector.append(next_state)
      
        # cumulative time (t + ∆t) // cumulative costs (t + ∆t) // cumulative budget utilization
        time = time + np.random.exponential(exp_rates[decision][next_state].loc[state])
        recovery_cost = recovery_cost + (fixed_cost_matrix[decision].loc[state] + time*cost_rate.loc[state])[0].astype(type('float', (float,), {})) # (1) [0] pandas core series to return numerical value (2) '.astype(type('float', (float,), {}))' to convert numpy.float64 into float
        budget_ut = budget_ut + fixed_cost_matrix[decision].loc[state].astype(type('float', (float,), {}))

        # Times and costs vector update
        time_vector_fst.append(time)
        cost_vector_fst.append(recovery_cost)
        budget_fst.append(budget_ut/total_budget)

        # State update
        state = next_state
      
      # recording time and cost values per running
      fst_policy_time.append(time_vector_fst[len(time_vector_fst)-1])
      fst_policy_cost.append(cost_vector_fst[len(cost_vector_fst)-1])
      fst_policy_bud.append(budget_fst[len(budget_fst)-1])

    if policies_comparison == 1:
      while  state < (len(optimal_policy)-10):
        # state transition
        decision = optimal_policy[state].astype(int) # optimal policy   
        next_state = state_generator(exp_rates[decision],state)
        
        current_state_vector.append(state)
        next_state_vector.append(next_state)
      
        # cumulative time (t + ∆t) // cumulative costs (t + ∆t) // cumulative budget utilization
        time = time + np.random.exponential(exp_rates[decision][next_state].loc[state])
        recovery_cost = recovery_cost + (fixed_cost_matrix[decision].loc[state] + time*cost_rate.loc[state])[0].astype(type('float', (float,), {})) # (1) [0] pandas core series to return numerical value (2) '.astype(type('float', (float,), {}))' to convert numpy.float64 into float
        budget_ut = budget_ut + fixed_cost_matrix[decision].loc[state].astype(type('float', (float,), {}))

        # Times and costs vector update
        time_vector_opt.append(time) # optimal policy
        cost_vector_opt.append(recovery_cost)
        budget_opt.append(budget_ut/total_budget)

        # State update
        state = next_state
      
      # recording time and cost values per running
      opt_policy_time.append(time_vector_opt[len(time_vector_opt)-1])
      opt_policy_cost.append(cost_vector_opt[len(cost_vector_opt)-1])
      opt_policy_bud.append(budget_opt[len(budget_opt)-1])

    if policies_comparison == 2:
      while  state < (len(optimal_policy)-10):
        # state transition
        decision = 4 # lowest cost policy
        next_state = state_generator(exp_rates[decision],state)
        
        current_state_vector.append(state)
        next_state_vector.append(next_state)
      
        # cumulative time (t + ∆t) // cumulative costs (t + ∆t) // cumulative budget utilization
        time = time + np.random.exponential(exp_rates[decision][next_state].loc[state])
        recovery_cost = recovery_cost + (fixed_cost_matrix[decision].loc[state] + time*cost_rate.loc[state])[0].astype(type('float', (float,), {})) # (1) [0] pandas core series to return numerical value (2) '.astype(type('float', (float,), {}))' to convert numpy.float64 into float
        budget_ut = budget_ut + fixed_cost_matrix[decision].loc[state].astype(type('float', (float,), {}))


        # Times and costs vector update
        time_vector_chp.append(time)
        cost_vector_chp.append(recovery_cost)
        budget_chp.append(budget_ut/total_budget)

        # State update
        state = next_state
      
      # recording time and cost values per running
      chp_policy_time.append(time_vector_chp[len(time_vector_chp)-1])
      chp_policy_cost.append(cost_vector_chp[len(cost_vector_chp)-1])
      chp_policy_bud.append(budget_chp[len(budget_chp)-1])

    #updating running
    n_of_interations = n_of_interations + 1
  policies_comparison = policies_comparison + 1

## Resilience assessment

In [ ]:
# Zobel metric
def res_zobel(time, control = 120, X = .9):
  T = control # Control time T = 120 Months
  X = .9 # Performance loss 100%
  resilience = 1 - ((X*time)/(2*T))
  return resilience

In [ ]:
# Zobel metric
def res_zobel(time, control = 120, X = .9):
  resilience = 1 - ((X*time)/(2*control))
  return resilience

## Confidence intervals function

In [ ]:
def CI(dataset, cl):
  LB = np.mean(dataset) - sps.norm.ppf(1-(1-cl)/2)*np.sqrt(np.var(dataset)/len(dataset))
  UB = np.mean(dataset) + sps.norm.ppf(1-(1-cl)/2)*np.sqrt(np.var(dataset)/len(dataset))
  return (LB,UB)

## Simulation summary

In [ ]:
# Summary cols

policies = ['Fastest policy', 'Optimal policy', 'Lowest-cost policy'] # policies vector
mean_time = [np.mean(fst_policy_time), np.mean(opt_policy_time), np.mean(chp_policy_time)]
mean_cost = [np.mean(fst_policy_cost), np.mean(opt_policy_cost), np.mean(chp_policy_cost)]
mean_budget= [np.mean(fst_policy_bud), np.mean(opt_policy_bud), np.mean(chp_policy_bud)]
std_cost = [np.std(fst_policy_cost), np.std(opt_policy_cost), np.std(chp_policy_cost)]
std_time = [np.std(fst_policy_time), np.std(opt_policy_time), np.std(chp_policy_time)]
resilience = [res_zobel(np.mean(fst_policy_time)), res_zobel(np.mean(opt_policy_time)), res_zobel(np.mean(chp_policy_time))]
time_LB = [CI(fst_policy_time, .95)[0], CI(opt_policy_time, .95)[0], CI(chp_policy_time, .95)[0]]
time_UB = [CI(fst_policy_time, .95)[1], CI(opt_policy_time, .95)[1], CI(chp_policy_time, .95)[1]]
cost_LB = [CI(fst_policy_cost, .95)[0], CI(opt_policy_cost, .95)[0], CI(chp_policy_cost, .95)[0]]
cost_UB = [CI(fst_policy_cost, .95)[1], CI(opt_policy_cost, .95)[1], CI(chp_policy_cost, .95)[1]]


sim_summary = pd.DataFrame({'Polcies': policies, 'Mean recovery time': mean_time, 'Time lower-bound':time_LB, 'Time Upper-bound': time_UB,
                                                 'Mean recovery cost': mean_cost, 'Cost lower-bound':cost_LB, 'Cost Upper-bound': cost_UB,
                                                 'Mean budget utilizacion (%)': mean_budget,
                                                 'Average resilience T* = 120 (Zobel)': resilience})